In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import numpy as np
import pickle
import numpy as np
import os
import pandas as pd

In [ ]:
colnames=['date','text', 't1', 't2', 't3', 't4', 't5','6']
data = pd.read_csv('/content/twitter_sample_tweets.csv', names=colnames, header=None, on_bad_lines='skip')
print(data.shape)

In [ ]:
print(data['text'].head(10000))

In [ ]:
import demoji
data = data.head(10000).applymap(lambda x: demoji.replace(str(x),' '))
print(data['text'])

In [ ]:
data['text'] = data['text'].str.replace('[#,@,&,/,:,~,`,!,$,%,^,*,(,),_,-,+,=,{,},|,\,:,;,",<,>,?,؟,/,↫]', ' ')
data['text'] = data['text'].str.replace('[', ' ')
data['text'] = data['text'].str.replace(']', ' ')
data['text'] = data['text'].str.replace('.', ' ')
data['text'] = data['text'].str.replace(',', ' ')
data['text'] = data['text'].str.replace('RT', ' ')
data['text'] = data['text'].str.replace(' و ', ' ')
data['text'] = data['text'].str.replace(' از ', ' ')
data['text'] = data['text'].str.replace(' که ', ' ')
data['text'] = data['text'].str.replace(' به ', ' ')
data['text'] = data['text'].str.replace(' برای ', ' ')
data['text'] = data['text'].str.replace(' از ', ' ')
data['text'] = data['text'].str.replace(' چه ', ' ')
data['text'] = data['text'].str.replace(' با ', ' ')
data['text'] = data['text'].str.replace(' ای ', ' ')
data['text'] = data['text'].str.replace(' بود ', ' ')
data['text'] = data['text'].str.replace(' در ', ' ')
data['text'] = data['text'].str.replace(' می ', ' ')
data['text'] = data['text'].str.replace(' هم ', ' ')
data['text'] = data['text'].str.replace(' ولی ', ' ')
data['text'] = data['text'].str.replace(' هر ', ' ')
data['text'] = data['text'].str.replace(' یه ', ' ')
data['text'] = data['text'].str.replace(' را ', ' ')
data['text'] = data['text'].str.replace(' رو ', ' ')
data['text'] = data['text'].str.replace(' از ', ' ')
data['text'] = data['text'].str.replace('  ', ' ')
print(data['text'].shape)

<ipython-input-5-292dd1c96a63>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace('[#,@,&,/,:,~,`,!,$,%,^,*,(,),_,-,+,=,{,},|,\,:,;,",<,>,?,؟,/,↫]', ' ')
<ipython-input-5-292dd1c96a63>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['text'] = data['text'].str.replace('[', ' ')
<ipython-input-5-292dd1c96a63>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['text'] = data['text'].str.replace(']', ' ')
<ipython-input-5-292dd1c96a63>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will

(2948707,)


In [ ]:
data['text'] = data['text'].str.replace('\d+', '')
print(data['text'][1])

In [ ]:
with open('/content/sample.txt', 'w' ,encoding="utf-8") as f:
    dfAsString = str(data['text'].to_list())
    print(len(dfAsString))
    f.write(dfAsString)

In [ ]:
file = open("/content/sample.txt", "r", encoding = "utf8")
lines = []
for i in file:
    lines.append(i)
print(lines)
data = ""
for i in lines:
    data = ' '. join(lines)
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space
data = data.split()
data = ' '.join(data)
print(len(data))
print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(len(data))
print(data[10])
data = data.replace("'", ' ')
import re
data = re.sub(r'\s*[A-Za-z]+\b', '' , data)
print(data)
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
print(data[10])
#print(len(data))

In [ ]:
pickle.dump(tokenizer, open('token.pkl', 'wb'))
seq = tokenizer.texts_to_sequences([data])[0]
print(seq)

In [ ]:
size = len(tokenizer.word_index) + 1
print(len(tokenizer.word_index))

print(tokenizer.word_index)


In [ ]:
sequences = []

for i in range(3, len(seq)):
    words = seq[i-3:i+1]
    sequences.append(words)

print("Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

In [ ]:
x= []
y = []
for i in sequences:
    x.append(i[0:3])
    y.append(i[3])
x = np.array(x)
y = np.array(y)

In [ ]:
print("Data: ", x[:10])
print("Response: ", y[:10])

In [ ]:
y = to_categorical(y, num_classes=size)
y[:5]

In [ ]:
model = Sequential()
model.add(Embedding(size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(size, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 10)             263400    
                                                                 
 lstm_2 (LSTM)               (None, 3, 1000)           4044000   
                                                                 
 lstm_3 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense_1 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_2 (Dense)             (None, 26340)             26366340  
                                                                 
Total params: 39,678,740
Trainable params: 39,678,740
Non-trainable params: 0
_________________________________________________________________


In [ ]:
checkpoint = ModelCheckpoint("words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(x, y, epochs=20, batch_size=64, callbacks=[checkpoint])

Epoch 1/20
2349/2349 [==============================] - ETA: 0s - loss: 8.5608
Epoch 1: loss improved from inf to 8.56079, saving model to next_words.h5
2349/2349 [==============================] - 787s 333ms/step - loss: 8.5608
Epoch 2/20
2349/2349 [==============================] - ETA: 0s - loss: 8.1240
Epoch 2: loss improved from 8.56079 to 8.12396, saving model to next_words.h5
2349/2349 [==============================] - 787s 335ms/step - loss: 8.1240
Epoch 3/20
2349/2349 [==============================] - ETA: 0s - loss: 7.6308
Epoch 3: loss improved from 8.12396 to 7.63080, saving model to next_words.h5
2349/2349 [==============================] - 800s 340ms/step - loss: 7.6308
Epoch 4/20
2349/2349 [==============================] - ETA: 0s - loss: 7.0223
Epoch 4: loss improved from 7.63080 to 7.02230, saving model to next_words.h5
2349/2349 [==============================] - 823s 351ms/step - loss: 7.0223
Epoch 5/20
2349/2349 [==============================] - ETA: 0s - loss: 

In [ ]:


model = load_model('words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict(model, tokenizer, text):
  sequence = tokenizer.texts_to_sequences([text])

  if len(sequence[0]) < 3:

    sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=3)
  else:
    sequence = np.array(sequence)

  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key, value in tokenizer.word_index.items():
    if value == preds:
      predicted_word = key
      break

  print(predicted_word)
  return predicted_word



In [ ]:
while(True):
  text = input("Enter your line: ")

  if text == "0":
      print("Execution completed.....")
      break

  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)

          Predict(model, tokenizer, text)

      except Exception as e:
        print("Error occurred: ",e)
        continue
